# Building and Testing the performance of different models for the "Occupation Quota" of a Parisian street

## ----------------------------------- Data loading and Preprocessing -----------------------------------

#### Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import time
import os
import tqdm.notebook as tq
import tensorflow as tf
import warnings
import matplotlib.image as mpimg
warnings.filterwarnings('ignore')
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from keras.layers.normalization import BatchNormalization

#### Connect to Google Drive to get access to data later on

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/'Dossier partagé avec Iheb BELGACEM'/BCG_Hackathon

/content/drive/.shortcut-targets-by-id/1yXfewQDE3qy07N5B0XsOVSfqgaaXT3R7/Dossier partagé avec Iheb BELGACEM/BCG_Hackathon


#### Load dataframe and start preprocessing

In [ ]:
data_path = "datasets/wash1.csv"
TARGET = "Taux d'occupation"
#TARGET = "Débit horaire"
dataframe = pd.read_csv(data_path)
dataframe.drop(["Unnamed: 0","index"], axis = 1 , inplace= True)
def set_date(df):
    df["Date et heure de comptage"] = pd.to_datetime(df["Date et heure de comptage"],format='%Y-%m-%d %H:%M:%S%z')
    return df
dataframe = set_date(dataframe)
dataframe.time = dataframe.time.apply(lambda x : int(x[:2]))
if TARGET == "Taux d'occupation": 
  dataframe.drop(["Débit horaire","mean_debit_horaire_past_week"], axis = 1 , inplace= True)
else : 
  dataframe.drop(["Taux d'occupation","mean_taux_occupation_past_week"], axis = 1 , inplace= True)
dataframe.drop(["Etat trafic","Libelle noeud amont","Libelle noeud aval","date"], axis = 1 , inplace= True)
dataframe

,Date et heure de comptage,Taux d'occupation,weekday,weekofyear,month,year,time,temp,dwpt,rhum,tavg,tmin,tmax,pres,mean_taux_occupation_past_week,taux_occupation_past_week,debit_horaire_past_week,taux_occupation_sae,retail_and_recreation_percent_change_from_baseline,confin_0,confin_1,confin_2,couvrefeu,ferie
0,2019-11-14 04:00:00+01:00,3.20167,3,46,11,2019,4,5.1,4.2,94.0,5.6,4.6,6.5,997.9,15.835636,5.89833,298.0,12.289109,96.0,1,0,0,0,0
1,2019-11-14 05:00:00+01:00,3.27389,3,46,11,2019,5,5.3,4.3,93.0,5.6,4.6,6.5,997.9,15.809154,2.29278,260.0,12.289109,96.0,1,0,0,0,0
2,2019-11-14 06:00:00+01:00,5.28222,3,46,11,2019,6,5.5,3.3,86.0,5.6,4.6,6.5,997.9,15.828470,6.15222,430.0,12.289109,96.0,1,0,0,0,0
3,2019-11-14 07:00:00+01:00,9.07722,3,46,11,2019,7,5.2,3.4,88.0,5.6,4.6,6.5,997.9,15.854842,7.03278,692.0,12.289109,96.0,1,0,0,0,0
4,2019-11-14 08:00:00+01:00,15.51111,3,46,11,2019,8,5.2,3.5,89.0,5.6,4.6,6.5,997.9,15.900355,12.87889,1128.0,12.289109,96.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9331,2020-11-29 20:00:00+01:00,13.09667,6,48,11,2020,20,1.0,-0.9,87.0,2.5,1.0,6.0,1023.0,5.704904,9.45389,500.0,72.933663,34.0,0,0,1,0,0
9332,2020-11-29 21:00:00+01:00,7.50778,6,48,11,2020,21,1.0,-0.9,87.0,2.5,1.0,6.0,1023.0,5.720698,4.75833,359.0,72.933663,34.0,0,0,1,0,0
9333,2020-11-29 22:00:00+01:00,4.38000,6,48,11,2020,22,2.0,0.1,87.0,2.5,1.0,6.0,1023.0,5.733952,2.91389,256.0,72.933663,34.0,0,0,1,0,0
9334,2020-11-29 23:00:00+01:00,2.11500,6,48,11,2020,23,2.0,0.1,87.0,2.5,1.0,6.0,1023.0,5.743793,1.89722,149.0,72.933663,34.0,0,0,1,0,0


In [ ]:
dataframe = dataframe.interpolate() #get rid of NaN values
#dataframe

In [ ]:
dataframe.columns

Index(['Date et heure de comptage', 'Taux d'occupation', 'weekday',
       'weekofyear', 'month', 'year', 'time', 'temp', 'dwpt', 'rhum', 'tavg',
       'tmin', 'tmax', 'pres', 'mean_taux_occupation_past_week',
       'taux_occupation_past_week', 'debit_horaire_past_week',
       'taux_occupation_sae',
       'retail_and_recreation_percent_change_from_baseline', 'confin_0',
       'confin_1', 'confin_2', 'couvrefeu', 'ferie'],
      dtype='object')

In [ ]:
feat_time = ['weekday','weekofyear', 'month', 'year', 'time']
feat_meteo_d = ['tmin', 'tmax']
feat_meteo_h = ['temp', 'dwpt', 'rhum']
feat_conf_fer = ['confin_0','confin_1', 'confin_2', 'couvrefeu', 'ferie']
feat_mean = ['mean_taux_occupation_past_week'] if TARGET =="Taux d'occupation" else ['mean_debit_horaire_past_week']
feat_google = ['retail_and_recreation_percent_change_from_baseline']
feat_covid = ['taux_occupation_sae']

In [ ]:
split_rate = 0.8
i_split = int(dataframe.shape[0] * split_rate)
df_train = dataframe[:i_split]
df_test = dataframe[i_split:]
tt = df_test['Date et heure de comptage']

### Model1 - LSTM


##### Libraries

In [ ]:
to_scale = ['temp', 'dwpt', 'rhum','tmin', 'tmax','taux_occupation_sae','retail_and_recreation_percent_change_from_baseline']
for s in to_scale : 
  dataframe[s] = preprocessing.scale(dataframe[s])
dataframe

,Date et heure de comptage,Taux d'occupation,weekday,weekofyear,month,year,time,temp,dwpt,rhum,tavg,tmin,tmax,pres,mean_taux_occupation_past_week,taux_occupation_past_week,debit_horaire_past_week,taux_occupation_sae,retail_and_recreation_percent_change_from_baseline,confin_0,confin_1,confin_2,couvrefeu,ferie
0,2019-11-14 04:00:00+01:00,3.20167,3,46,11,2019,4,-1.122753,-0.546167,1.197244,5.6,-0.785542,-1.407489,997.9,15.835636,5.89833,298.0,-0.577140,1.226208,1,0,0,0,0
1,2019-11-14 05:00:00+01:00,3.27389,3,46,11,2019,5,-1.094279,-0.526431,1.146397,5.6,-0.785542,-1.407489,997.9,15.809154,2.29278,260.0,-0.577140,1.226208,1,0,0,0,0
2,2019-11-14 06:00:00+01:00,5.28222,3,46,11,2019,6,-1.065805,-0.723800,0.790472,5.6,-0.785542,-1.407489,997.9,15.828470,6.15222,430.0,-0.577140,1.226208,1,0,0,0,0
3,2019-11-14 07:00:00+01:00,9.07722,3,46,11,2019,7,-1.108516,-0.704063,0.892165,5.6,-0.785542,-1.407489,997.9,15.854842,7.03278,692.0,-0.577140,1.226208,1,0,0,0,0
4,2019-11-14 08:00:00+01:00,15.51111,3,46,11,2019,8,-1.108516,-0.684326,0.943011,5.6,-0.785542,-1.407489,997.9,15.900355,12.87889,1128.0,-0.577140,1.226208,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9331,2020-11-29 20:00:00+01:00,13.09667,6,48,11,2020,20,-1.706473,-1.552750,0.841319,2.5,-1.436873,-1.473547,1023.0,5.704904,9.45389,500.0,1.397328,-0.705718,0,0,1,0,0
9332,2020-11-29 21:00:00+01:00,7.50778,6,48,11,2020,21,-1.706473,-1.552750,0.841319,2.5,-1.436873,-1.473547,1023.0,5.720698,4.75833,359.0,1.397328,-0.705718,0,0,1,0,0
9333,2020-11-29 22:00:00+01:00,4.38000,6,48,11,2020,22,-1.564102,-1.355381,0.841319,2.5,-1.436873,-1.473547,1023.0,5.733952,2.91389,256.0,1.397328,-0.705718,0,0,1,0,0
9334,2020-11-29 23:00:00+01:00,2.11500,6,48,11,2020,23,-1.564102,-1.355381,0.841319,2.5,-1.436873,-1.473547,1023.0,5.743793,1.89722,149.0,1.397328,-0.705718,0,0,1,0,0


###### Prepare data for THIS model, separate training and test set

In [ ]:
def lstm_test_transform(x_data,e,num_steps) : 
  x,y = list(),list()
  for i in range(x_data.index[0], x_data.index[-1], 24*6) : 
    end_ix = i+num_steps
    if (end_ix + 24*6)>= x_data.index[-1] : 
      break
    seq_x = x_data[e].loc[i:end_ix].values
    seq_y = x_data.loc[end_ix:end_ix + 24*6,TARGET]
    x.append(seq_x)
    y.append(seq_y)
  x_array = np.array(x)
  y_array = np.array(y)
  return x_array , y_array
def lstm_data_transform(x_data,e,num_steps) : 
  x,y = list(),list()
  for i in range(x_data.shape[0]) : 
    end_ix = i+num_steps
    if (end_ix + 24*6)>= x_data.shape[0] : 
      break
    seq_x = x_data[e][i:end_ix].values
    seq_y = x_data.loc[end_ix:end_ix + 24*6,TARGET]
    x.append(seq_x)
    y.append(seq_y)
  x_array = np.array(x)
  y_array = np.array(y)
  return x_array , y_array

In [ ]:
model_name = "LSTM"
if model_name not in os.listdir(os.getcwd()) : 
  os.mkdir(model_name)
  print('Created Folder')
experiments = [
               [TARGET]+feat_meteo_d+feat_conf_fer,
               [TARGET] +feat_meteo_d+feat_google,
               [TARGET] +feat_meteo_d+feat_covid, 
               [TARGET] + feat_time+feat_meteo_d+feat_conf_fer+feat_covid+feat_google]
count = 0
resume = []

for NB_DAYS in [4,8] : 
  for nb_lstm in [40]:
    for dropout in [0.001]:
      for e in tq.tqdm(experiments) : 
        print(e)
        timestep = datetime.datetime.now().strftime('%Y-%m-%d %H_%M_%S')
        os.mkdir(os.path.join(os.getcwd(),model_name,timestep))
        x_train, y_train = lstm_data_transform(df_train,e,num_steps = 24*7*NB_DAYS)
        x_test, y_test = lstm_test_transform(df_test,e, num_steps = 24*7*NB_DAYS)
        y_test = y_test.flatten()
        model = Sequential()
        model.add(LSTM(nb_lstm,input_shape = (x_train.shape[1],x_train.shape[2]), return_sequences=True))
        model.add(Dropout(dropout))
        model.add(LSTM(nb_lstm))
        model.add(Dropout(dropout))
        model.add(Dense(145,activation = 'relu'))
        model.summary()
        model.compile(loss = 'mse', optimizer = 'adam')
        model.summary()
        epochs = 300
        es =tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode ='min',verbose = 1, patience = 5)
        model.fit(x_train, y_train, epochs = epochs, verbose = 1, validation_split=0.15, callbacks = [es], batch_size = 1024)
        pred = model.predict(x_test).flatten()
        mse_tot = mean_squared_error(y_test,pred)
        mse_first = mean_squared_error(y_test[:24*6],pred[:24*6])
        record = {'name' : timestep , 'features' : e, "mse_tot":mse_tot , "mse_first" :mse_first ,"nb_lstm" : nb_lstm ,"nb_days":NB_DAYS, "dropout" : dropout, "path": os.path.join(os.getcwd(), model_name, timestep)}
        resume.append(record)
        pd.DataFrame.from_records([record]).to_csv(os.path.join(os.getcwd(),model_name,timestep,timestep+".csv"))
        fig = plt.figure(count)
        count = count + 1
        plt.plot(y_test , color = 'r',label = 'real values')
        plt.plot(pred , color ='b',label = 'prediected values')
        plt.title(model_name + "  ||  The MSE_tot: {}".format(mse_tot))
        fig.savefig(os.path.join(os.getcwd(),model_name,timestep,'total'+timestep+'.jpeg'))
        fig = plt.figure(count)
        count = count +1 
        plt.plot(tt[:24*6], y_test[:24*6] , color = 'r',label = 'real values')
        plt.plot(tt[:24*6], pred[:24*6] , color = 'b',label = 'predicted values')
        plt.title(model_name + "  ||  The MSE_first_6days: {}".format(mse_first))
        fig.savefig(os.path.join(os.getcwd(), model_name,timestep,'first'+timestep+'.jpeg'))
        pd.DataFrame.from_records(resume).sort_values('mse_tot').to_csv(model_name+".csv")
pd.DataFrame.from_records(resume).sort_values('mse_tot').to_csv(model_name+".csv")
resume = pd.read_csv(model_name+'.csv')
del resume['Unnamed: 0']

["Taux d'occupation", 'tmin', 'tmax', 'confin_0', 'confin_1', 'confin_2', 'couvrefeu', 'ferie']
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 672, 40)           7840      
_________________________________________________________________
dropout_12 (Dropout)         (None, 672, 40)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 40)                12960     
_________________________________________________________________
dropout_13 (Dropout)         (None, 40)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 145)               5945      
Total params: 26,745
Trainable params: 26,745
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_6

In [ ]:
resume

In [ ]:
for i in range(10) : 
  print(i)
  print("MSE_tot: {}    ||     Features: {}".format( str(resume.loc[i,"mse_tot"]), resume.loc[i,'features']))

In [ ]:
for i in range(5) : 
  print(resume.loc[i])
  path = resume.loc[i,"path"]
  for file in os.listdir(path) :
    if '.jpeg' in file : 
      img = mpimg.imread(os.path.join(resume.loc[i,'path'],file))
      plt.imshow(img)
      plt.show()